In [ ]:
import os
import re
import csv
import json
import zipfile
import urllib.request
import tempfile
import unicodedata
from typing import Dict, List, Optional, Tuple

import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt
from matplotlib import font_manager
import numpy as np

# IMPORTACIÓN FALTANTE: requests es necesaria para el GeoJSON
import requests

# ------------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# ------------------------------------------------------------------------------
# Recuperamos variables inyectadas por app.py
estado_seleccionado = locals().get("ESTADO_SELECCIONADO", "Aguascalientes")
palette = locals().get("active_palette", ["#0e1c2c", "#d9d9d9"])
active_font = locals().get("active_font", "sans-serif")

# Colores dinámicos adaptados de la paleta de Streamlit
COLOR_ESTADO = palette[0]
COLOR_OTROS = '#d9d9d9'  # Gris suave fijo para el fondo del mapa
COLOR_BORDE = '#8a8a8a'  # Borde de las entidades

# URLs de Datos
ZIP_URL = "https://www.inegi.org.mx/contenidos/programas/pibent/2018/datosabiertos/conjunto_de_datos_piber_csv.zip"
GEOJSON_URL = "https://raw.githubusercontent.com/pato-gg/INEGI-GeoJSON/main/Entidades_M%C3%A9xico.json"

# Nombres y alias
ALIASES = {
    'estado de mexico':'México','edomex':'México', 'cdmx':'Ciudad de México',
    'distrito federal':'Ciudad de México','ciudad de mexico':'Ciudad de México',
    'veracruz':'Veracruz de Ignacio de la Llave','queretaro':'Querétaro',
    'san luis potosi':'San Luis Potosí', 'nacional':'Estados Unidos Mexicanos',
    'coahuila': 'Coahuila de Zaragoza', 'michoacán': 'Michoacán de Ocampo',
}
CODE2NAME = {
    'ags':'Aguascalientes','bc':'Baja California','bcs':'Baja California Sur','camp':'Campeche',
    'coah':'Coahuila de Zaragoza','col':'Colima','chis':'Chiapas','chih':'Chihuahua','cdmx':'Ciudad de México',
    'dgo':'Durango','gto':'Guanajuato','gro':'Guerrero','hgo':'Hidalgo','jal':'Jalisco','mex':'México',
    'mich':'Michoacán de Ocampo','mor':'Morelos','nay':'Nayarit','nl':'Nuevo León','oax':'Oaxaca',
    'pue':'Puebla','qro':'Querétaro','qr':'Quintana Roo','slp':'San Luis Potosí','sin':'Sinaloa','son':'Sonora',
    'tab':'Tabasco','tamps':'Tamaulipas','tlax':'Tlaxcala','ver':'Veracruz de Ignacio de la Llave',
    'yuc':'Yucatán','zac':'Zacatecas','nac':'Estados Unidos Mexicanos'
}
FALLBACK_CPV_2020 = {
    "Estados Unidos Mexicanos": 126014024, "Aguascalientes": 1425607, "Baja California": 3769020,
    "Baja California Sur": 798447, "Campeche": 928363, "Coahuila de Zaragoza": 3146771,
    "Colima": 731391, "Chiapas": 5543828, "Chihuahua": 3801487, "Ciudad de México": 9209944,
    "Durango": 1832650, "Guanajuato": 6166934, "Guerrero": 3648096, "Hidalgo": 3082841,
    "Jalisco": 8348151, "México": 16992418, "Michoacán de Ocampo": 4876008, "Morelos": 1971520,
    "Nayarit": 1235457, "Nuevo León": 5784442, "Oaxaca": 4132143, "Puebla": 6583278,
    "Querétaro": 2279632, "Quintana Roo": 1871986, "San Luis Potosí": 2822255,
    "Sinaloa": 3271264, "Sonora": 2944840, "Tabasco": 2402599, "Tamaulipas": 3527738,
    "Tlaxcala": 1342977, "Veracruz de Ignacio de la Llave": 8112505, "Yucatán": 2320898,
    "Zacatecas": 1622138
}

# ------------------------------------------------------------------------------
# 2. UTILIDADES DE DATOS
# ------------------------------------------------------------------------------
# (Se mantienen funciones auxiliares limpias)
def strip_accents(s: str) -> str:
    return ''.join(c for c in unicodedata.normalize('NFD', s or '') if unicodedata.category(c) != 'Mn')
def norm(s: str) -> str:
    s0 = (s or '').strip().lower()
    s0 = re.sub(r'\s+', ' ', s0)
    return strip_accents(s0)
def normalize_entity_name(name: str) -> str:
    base = norm(name)
    if base in ALIASES: return ALIASES[base]
    for full in CODE2NAME.values():
        if norm(full) == base: return full
    return (name or '').strip()
def download(url: str, timeout=180) -> bytes:
    req = urllib.request.Request(url, headers={"User-Agent": "Streamlit/1.0 (pibe-map)"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        return r.read()
def find_index_file(extract_dir: str):
    pat = re.compile(r'i[íi]?ndice\.csv$', re.I)
    for root, _, files in os.walk(extract_dir):
        for fn in files:
            if pat.search(fn): return os.path.join(root, fn)
    return None
def read_index_mapping(index_csv: str):
    mapping = {}
    for enc in ('utf-8', 'latin-1'):
        try:
            with open(index_csv, 'r', encoding=enc, errors='ignore', newline='') as f:
                reader = csv.reader(f)
                for row in reader:
                    if len(row) >= 2:
                        k, v = row[0].strip(), row[1].strip()
                        if k and v: mapping[k] = v
            if mapping: break
        except Exception: continue
    return mapping
def find_state_file(mapping, state_name: str):
    target = normalize_entity_name(state_name)
    for k in mapping.keys():
        m = re.search(r'entidad_([a-z]+)20', k.lower())
        ent = CODE2NAME.get(m.group(1)) if m else None
        if ent and norm(ent) == norm(target): return k
    return None
def _to_float(x):
    s = str(x).strip().replace(',', '')
    try: return float(s)
    except Exception: return None
def detect_year_columns(cols):
    return sorted(set(int(re.search(r'(19|20)\d{2}', str(c)).group(0)) for c in cols if re.search(r'(19|20)\d{2}', str(c))))
def find_year_column(cols, year: int):
    return next((c for c in cols if str(year) in str(c)), None)

# Funciones de caché
@st.cache_data(show_spinner=False)
def get_pib_data_from_zip(zip_url: str, estado: str):
    workdir = tempfile.mkdtemp()
    zip_path = os.path.join(workdir, "piber_csv.zip")
    try:
        r = urllib.request.Request(zip_url, headers={"User-Agent": "Streamlit/1.0 (pibe-map)"})
        data_zip = urllib.request.urlopen(r, timeout=180).read()
        with open(zip_path, 'wb') as f: f.write(data_zip)
        extract_dir = os.path.join(workdir, "extracted")
        with zipfile.ZipFile(zip_path, 'r') as z: z.extractall(extract_dir)

        idx_path = next((os.path.join(r, fn) for r, _, fs in os.walk(extract_dir)
                         for fn in fs if re.search(r'i[íi]?ndice\.csv$', fn, re.I)), None)
        if not idx_path: raise FileNotFoundError("No se encontró índice CSV.")

        mapping = {}
        with open(idx_path, 'r', encoding='latin-1') as f:
             mapping = {row[0]: row[1] for row in csv.reader(f) if len(row) >= 2}

        fname_code = find_state_file(mapping, estado)
        if not fname_code: raise FileNotFoundError(f"No se encontró archivo para '{estado}'.")

        basename = os.path.basename(fname_code)
        target_csv = next((os.path.join(r, fn) for r, _, fs in os.walk(extract_dir)
                           for fn in fs if fn == basename), None)
        if not target_csv: raise FileNotFoundError(f"No se ubicó '{basename}' en la extracción.")

        df = pd.read_csv(target_csv, encoding='latin-1', low_memory=False)
        mask = df.iloc[:,0].astype(str).str.contains(r'Producto\s*interno\s*bruto', case=False, regex=True, na=False)
        if not mask.any(): raise ValueError("Fila 'Producto interno bruto' no encontrada.")
        row = df[mask].iloc[0]
        years = detect_year_columns(df.columns)
        if not years: raise ValueError("No se detectaron columnas de años.")
        last_year = max(years)
        col_last = find_year_column(df.columns, last_year)
        val_last = _to_float(row[col_last])
        val_prev = None
        if len(years) > 1:
            prev_year = sorted(years)[-2]
            col_prev = find_year_column(df.columns, prev_year)
            if col_prev: val_prev = _to_float(row[col_prev])

        estado_norm = normalize_entity_name(estado)
        return last_year, val_last, val_prev, estado_norm

    except Exception as e:
        raise RuntimeError(f"Fallo en el pipeline del PIBE: {e}")

@st.cache_data(show_spinner=False)
def get_poblacion_total_2020(estado: str) -> int:
    estado_norm = normalize_entity_name(estado)
    if estado_norm in FALLBACK_CPV_2020: return FALLBACK_CPV_2020[estado_norm]
    for k in FALLBACK_CPV_2020.keys():
        if norm(k) == norm(estado_norm): return FALLBACK_CPV_2020[k]
    return 0

@st.cache_data(show_spinner=False)
def get_geojson(url: str):
    """Descarga el GeoJSON de entidades (CORREGIDO: usa requests)."""
    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        # Se lanza el RuntimeError, pero con el requests importado ahora funcionará
        raise RuntimeError(f"Fallo al descargar GeoJSON: {e}")

# ------------------------------------------------------------------------------
# 3. MAPA (MATPLOTLIB) - Modificado para integrar colores y Streamlit
# ------------------------------------------------------------------------------
def format_pesos_millones(x: float) -> str:
    return f"{x:,.2f}"
def format_miles(n: int) -> str:
    return f"{n:,}"

def guess_name_prop(props: dict):
    candidates = ['NOMGEO','NOM_ENT','nom_ent','nomgeo','nombre','state_name','Entidad','ENTIDAD','Name','name']
    for k in candidates:
        if k in props: return k
    return next((k for k, v in props.items() if isinstance(v, str) and 3 <= len(v) <= 40), None)

def draw_mex_map_highlight(geojson_data: dict, estado_objetivo: str, pib_val: float, year: int, poblacion: int):
    # Intentamos establecer la fuente dinámica
    plt.rcParams['font.family'] = active_font

    feats = geojson_data.get('features', [])
    if not feats: raise ValueError("GeoJSON sin 'features'.")
    name_key = guess_name_prop(feats[0].get('properties', {}))
    if not name_key: raise ValueError("No pude detectar el campo de nombre de las entidades.")

    def geom_iter(feature):
        g = feature.get('geometry', {})
        gtype = g.get('type')
        coords = g.get('coordinates', [])
        if gtype == 'Polygon':
            for ring in coords: yield ring
        elif gtype == 'MultiPolygon':
            for poly in coords:
                for ring in poly: yield ring

    xmin = ymin = 1e9
    xmax = ymax = -1e9
    estados_data = []
    for f in feats:
        nombre = f.get('properties', {}).get(name_key, '')
        estados_data.append((nombre, f))
        for ring in geom_iter(f):
            if len(ring) >= 2:
                xs, ys = zip(*ring)
                xmin, xmax = min(xmin, min(xs)), max(xmax, max(xs))
                ymin, ymax = min(ymin, min(ys)), max(ymax, max(ys))

    # Figura de Matplotlib
    fig, ax = plt.subplots(figsize=(8.5, 6.2), dpi=140)
    ax.set_facecolor("white")
    ax.axis('off')

    # Fondo gris (estados no seleccionados)
    for nombre, f in estados_data:
        for ring in geom_iter(f):
            xs, ys = zip(*ring)
            ax.fill(xs, ys, facecolor=COLOR_OTROS, edgecolor=COLOR_BORDE, linewidth=0.6, zorder=1)

    # Estado destacado (usa COLOR_ESTADO de la paleta de la app)
    estado_norm = normalize_entity_name(estado_objetivo)
    encontrado = False
    for nombre, f in estados_data:
        if norm(nombre) == norm(estado_norm):
            for ring in geom_iter(f):
                xs, ys = zip(*ring)
                ax.fill(xs, ys, facecolor=COLOR_ESTADO, edgecolor=COLOR_ESTADO, linewidth=0.8, zorder=2)
            encontrado = True
            break
    if not encontrado:
        raise ValueError(f"No pude ubicar '{estado_objetivo}' en el GeoJSON.")

    # Límites del mapa
    dx, dy = (xmax - xmin), (ymax - ymin)
    ax.set_xlim(xmin - 0.02*dx, xmax + 0.02*dx)
    ax.set_ylim(ymin - 0.02*dy, ymax + 0.02*dy)
    ax.set_aspect('equal', adjustable='box')

    # Título principal del mapa (CENTRADO)
    ax.set_title(
        f"PIB Estatal y Población — {estado_norm}",
        fontsize=16,
        fontfamily=active_font,
        fontweight='bold',
        x=0.5, y=1.05 # Posición centrada
    )

    # Recuadro de datos (usa COLOR_ESTADO)
    box_txt = (
        f"Población: {format_miles(poblacion)}\n"
        f"PIB: "
        f"${format_pesos_millones(pib_val)} millones)"
    )
    ax.text(
        0.98, 0.98, box_txt, transform=ax.transAxes, ha='right', va='top',
        fontsize=11, color=COLOR_ESTADO,
        bbox=dict(boxstyle='round,pad=0.55', fc='white', ec=COLOR_ESTADO, lw=1.6)
    )
    plt.tight_layout()

    return fig

# ------------------------------------------------------------------------------
# 4. FLUJO PRINCIPAL DE STREAMLIT
# ------------------------------------------------------------------------------

st.markdown("### 🗺️ Análisis Geo-Económico Estatal")

if not estado_seleccionado:
    st.info("Selecciona un estado para ver el análisis de PIB y Población.")
else:
    # --- 1. Obtener datos ---
    try:
        with st.spinner(f"Cargando datos del PIBE, población y geografía de {estado_seleccionado}..."):
            year, pib_val, pib_prev, estado_pib_norm = get_pib_data_from_zip(ZIP_URL, estado_seleccionado)
            poblacion = get_poblacion_total_2020(estado_pib_norm)
            geojson_data = get_geojson(GEOJSON_URL)

    except Exception as e:
        # Si hay error en la carga, lo mostramos y detenemos el flujo.
        st.error(f"❌ Error al obtener datos: {e}")
        st.warning("Verifica la conexión a INEGI o el nombre del Estado. No se puede generar la gráfica sin los datos.")
        st.stop()

    # --- 2. Preparar métricas ---
    var_pct = None
    if pib_prev and pib_prev != 0:
        var_pct = ((pib_val - pib_prev) / pib_prev) * 100.0

    st.subheader(f"Datos de {estado_pib_norm}")

    col1, col2, col3 = st.columns(3)

    with col1:
        st.metric(
            f"PIB (Millones MXN)",
            f"${pib_val:,.2f}",
            f"{var_pct:.2f} % vs año anterior" if var_pct is not None else None,
            delta_color="inverse" if var_pct and var_pct < 0 else "normal",
            help="Producto Interno Bruto (base 2018)."
        )
    with col2:
        st.metric(
            "Población Total",
            f"{poblacion:,} hab."
        )
    with col3:
        pib_pc = (pib_val * 1_000_000 / poblacion) if poblacion > 0 else 0
        st.metric(
            f"PIB Per Cápita",
            f"${pib_pc:,.0f}"
        )

    st.divider()

    # --- 3. Dibujar mapa con Matplotlib y mostrar en Streamlit ---
    try:
        fig_map = draw_mex_map_highlight(geojson_data, estado_seleccionado, pib_val, year, poblacion)

        # Guardar la figura como buffer de bytes
        buf = io.BytesIO()
        fig_map.savefig(buf, format='png', bbox_inches='tight')
        plot_bytes = buf.getvalue()
        buf.close()

        # Mostrar la figura de Matplotlib en Streamlit
        st.pyplot(fig_map)

        st.download_button(
            label="Descargar Mapa",
            data=plot_bytes,
            file_name=f"mapa_PIB_{estado_seleccionado}.png",
            mime="image/png"
        )
        st.caption("Fuente: INEGI y Censo de Población y Vivienda.")
        plt.close(fig_map) # Limpiar la figura de Matplotlib

    except Exception as e:
        st.error(f"❌ Error al generar el mapa de Matplotlib: {e}")

ModuleNotFoundError: No module named 'streamlit'